In [2]:
import sys
sys.path.append('../')

In [19]:
import torch
import numpy as np
import albumentations as A
import matplotlib.pyplot as plt
from skimage.util import random_noise
from torchvision import transforms

from src.torch.dataset import data_split
from src.config import prepare_config
from src.visualize.plots import show_images

In [5]:
train_dataloader, val_dataloader, test_dataloader = data_split(
    train_batch_size=1,
    val_batch_size=1,
    test_batch_size=1,
    train_transforms=None,
    test_transforms=None,
    train_augm=None,
    test_augm=None,
    download=True,
    random_state=42,
)

100%|██████████| 170498071/170498071 [00:42<00:00, 4026063.21it/s]


Extracting ./data/CIFAR10/cifar-10-python.tar.gz to ./data/CIFAR10
Files already downloaded and verified


In [11]:
MEAN, STD = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)


class GaussianNoise(object):
    def __init__(self, mean=0, std=1):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean

    def __repr__(self):
        return self.__class__.__name__ + "(mean={0}, std={1})".format(
            self.mean, self.std
        )


class PepperNoise(object):
    def __init__(self, amount=0.1):
        self.amount = amount

    def __call__(self, tensor):
        return torch.tensor(random_noise(tensor, mode="pepper", amount=self.amount))

    def __repr__(self):
        return self.__class__.__name__ + "(amount={0})".format(self.amount)


def get_train_transforms():
    trans = [
        transforms.Resize((32, 32)),
        transforms.RandomResizedCrop(32, scale=(0.25, 1.0), ratio=(1.0, 1.0)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(15),
        transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2)),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.ToTensor(),
        transforms.Normalize(MEAN, STD),
        transforms.RandomApply([GaussianNoise(0, 0.05)], p=0.25),
    ]
    return transforms.Compose(trans)


def get_val_transforms():
    trans = [
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize(MEAN, STD),
    ]
    return transforms.Compose(trans)


In [12]:
image = train_dataloader.dataset[0][0]

In [13]:
train_tf = get_train_transforms()
test_tf = get_val_transforms()

In [23]:
_image = np.asarray(image)
transformed_image = train_tf(image).numpy().transpose(2, 1, 0)

In [27]:
show_images(images={})

TypeError: show_images() got an unexpected keyword argument '_image'